In [ ]:
# Análise da Taxa de Desistência no Ensino Superior Brasileiro
# Autor: [Seu Nome]
# Data: [Data]

# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os

# Configurações para visualizações de alta qualidade
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
sns.set_style('whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']  # Use the default Matplotlib font


# Criando diretório para salvar figuras se não existir
if not os.path.exists('figuras'):
    os.makedirs('figuras')

# Carregando os dados
print("Carregando dados de trajetória acadêmica...")
df = pd.read_parquet("data/indicadores_trajetoria/indicadores_trajetoria_educacao_superior_all.parquet")

# Exibindo informações básicas sobre o dataset
print(f"Dimensões do DataFrame: {df.shape}")
print("Tipos de dados:")
print(df.dtypes.value_counts())

# Verificando valores ausentes
missing_data = df.isnull().sum().sort_values(ascending=False)
missing_percent = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
missing = pd.concat([missing_data, missing_percent], axis=1, 
                   keys=['Valores Ausentes', 'Porcentagem'])
print("Top 10 colunas com valores ausentes:")
print(missing[missing['Valores Ausentes'] > 0].head(10))

# Estatísticas básicas das taxas de desistência
print("Estatísticas sobre taxas de desistência:")
dropout_stats = df[['Taxa de Desistência Acumulada - TDA', 'Taxa de Desistência Anual - TADA']].describe()
print(dropout_stats)

1. Limpeza e Preparação dos Dados
Vamos continuar com a limpeza e preparação dos dados:

In [ ]:
# 1. Limpeza e Preparação dos Dados

# 1.1 Verificando a consistência dos dados
print("\nVerificando consistência dos dados...")
print(f"Anos de ingresso únicos: {sorted(df['Ano de Ingresso'].unique())}")
print(f"Anos de referência únicos: {sorted(df['Ano de Referência'].unique())}")

# 1.2 Criando variáveis derivadas úteis para a análise
print("\nCriando variáveis derivadas...")

# Tempo no curso (ano de referência - ano de ingresso)
df['Tempo no Curso'] = df['Ano de Referência'] - df['Ano de Ingresso']

# Identificando categorias administrativas
cat_admin_map = {
    1.0: 'Pública Federal',
    2.0: 'Pública Estadual',
    3.0: 'Pública Municipal',
    4.0: 'Privada com fins lucrativos',
    5.0: 'Privada sem fins lucrativos',
    7.0: 'Especial'
}
df['Categoria Administrativa Descrição'] = df['Categoria Administrativa'].map(cat_admin_map)

# Identificando organizações acadêmicas
org_acad_map = {
    1.0: 'Universidade',
    2.0: 'Centro Universitário',
    3.0: 'Faculdade',
    4.0: 'Instituto Federal',
    5.0: 'Centro Federal de Educação Tecnológica'
}
df['Organização Acadêmica Descrição'] = df['Organização Acadêmica'].map(org_acad_map)

# Identificando graus acadêmicos
grau_acad_map = {
    1.0: 'Bacharelado',
    2.0: 'Licenciatura',
    3.0: 'Tecnológico'
}
df['Grau Acadêmico Descrição'] = df['Grau Acadêmico'].map(grau_acad_map)

# Identificando modalidades de ensino
mod_ensino_map = {
    1.0: 'Presencial',
    2.0: 'Ensino a Distância'
}
df['Modalidade de Ensino Descrição'] = df['Modalidade de Ensino'].map(mod_ensino_map)

# Identificando regiões geográficas
regiao_map = {
    1.0: 'Norte',
    2.0: 'Nordeste',
    3.0: 'Sudeste',
    4.0: 'Sul',
    5.0: 'Centro-Oeste'
}
df['Região Geográfica'] = df['Código da Região Geográfica do Curso'].map(regiao_map)

# 1.3 Removendo colunas redundantes ou com pouca relevância para análise
colunas_para_manter = [
    'Código da Instituição', 'Nome da Instituição', 
    'Categoria Administrativa', 'Categoria Administrativa Descrição',
    'Organização Acadêmica', 'Organização Acadêmica Descrição',
    'Nome do Curso de Graduação', 
    'Grau Acadêmico', 'Grau Acadêmico Descrição',
    'Modalidade de Ensino', 'Modalidade de Ensino Descrição',
    'Nome da área do Curso segundo a classificação CINE BRASIL',
    'Nome da Grande Área do Curso segundo a classificação CINE BRASIL',
    'Região Geográfica',
    'Ano de Ingresso', 'Ano de Referência', 'Tempo no Curso',
    'Prazo de Integralização em Anos',
    'Quantidade de Ingressantes no Curso',
    'Quantidade de Permanência no Curso no ano de referência',
    'Quantidade de Concluintes no Curso no ano de referência',
    'Quantidade de Desistência no Curso no ano de referência',
    'Taxa de Permanência - TAP',
    'Taxa de Conclusão Acumulada - TCA',
    'Taxa de Desistência Acumulada - TDA',
    'Taxa de Conclusão Anual - TCAN',
    'Taxa de Desistência Anual - TADA'
]

df_clean = df[colunas_para_manter].copy()
print(f"\nDataset após remoção de colunas redundantes: {df_clean.shape}")

# 1.4 Verificando valores anômalos ou incorretos nas taxas
# As taxas devem estar entre 0 e 100
print("\nVerificando valores anômalos nas taxas...")
taxas_colunas = ['Taxa de Permanência - TAP', 'Taxa de Conclusão Acumulada - TCA', 
                 'Taxa de Desistência Acumulada - TDA', 'Taxa de Conclusão Anual - TCAN', 
                 'Taxa de Desistência Anual - TADA']

for col in taxas_colunas:
    anomalias = ((df_clean[col] < 0) | (df_clean[col] > 100)).sum()
    print(f"Valores anômalos em {col}: {anomalias}")

# 1.5 Verificando a soma das taxas acumuladas (deve ser 100%)
df_clean['Soma das Taxas'] = df_clean['Taxa de Permanência - TAP'] + \
                           df_clean['Taxa de Conclusão Acumulada - TCA'] + \
                           df_clean['Taxa de Desistência Acumulada - TDA']

# Verificando se a soma é aproximadamente 100
tolerancia = 0.01  # Tolerância de 0.01% para erros de arredondamento
inconsistencias = ((df_clean['Soma das Taxas'] < 100-tolerancia) | 
                   (df_clean['Soma das Taxas'] > 100+tolerancia)).sum()
print(f"Registros com soma de taxas fora de 100% (±{tolerancia}%): {inconsistencias}")

# Removendo a coluna auxiliar após a verificação
df_clean.drop('Soma das Taxas', axis=1, inplace=True)

# Salvando o dataset limpo
df_clean.to_parquet('data/processed/trajetoria_academica_limpo.parquet')
print("\nDataset limpo salvo em 'data/processed/trajetoria_academica_limpo.parquet'")

2. Análise Exploratória dos Dados
Agora vamos realizar uma análise exploratória detalhada:

In [ ]:
# 2. Análise Exploratória dos Dados

# Carregando o dataset limpo (caso esteja executando em uma nova sessão)
# df_clean = pd.read_parquet('data/processed/trajetoria_academica_limpo.parquet')

# 2.1 Distribuição da taxa de desistência acumulada
plt.figure(figsize=(10, 6))
sns.histplot(df_clean['Taxa de Desistência Acumulada - TDA'], kde=True)
plt.title('Distribuição da Taxa de Desistência Acumulada', fontsize=12)
plt.xlabel('Taxa (%)')
plt.ylabel('Frequência')
plt.grid(True, linestyle='--', alpha=0.7)
plt.savefig('figuras/distribuicao_taxa_desistencia.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.2 Análise da taxa de desistência por tempo no curso
# Agrupando por tempo no curso e calculando a média da taxa
evolucao_tempo = df_clean.groupby('Tempo no Curso')['Taxa de Desistência Acumulada - TDA'].agg(
    ['mean', 'std', 'count']).reset_index()
evolucao_tempo.columns = ['Tempo no Curso', 'Taxa Média', 'Desvio Padrão', 'Contagem']

plt.figure(figsize=(12, 7))
sns.lineplot(x='Tempo no Curso', y='Taxa Média', data=evolucao_tempo, marker='o', linewidth=2)
plt.fill_between(evolucao_tempo['Tempo no Curso'], 
                 evolucao_tempo['Taxa Média'] - evolucao_tempo['Desvio Padrão'],
                 evolucao_tempo['Taxa Média'] + evolucao_tempo['Desvio Padrão'], 
                 alpha=0.3)
plt.title('Evolução da Taxa de Desistência Acumulada ao Longo do Tempo no Curso', fontsize=14)
plt.xlabel('Anos no Curso', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(evolucao_tempo['Tempo no Curso'].unique())
plt.savefig('figuras/evolucao_desistencia_tempo.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.3 Análise por variáveis categóricas
# 2.3.1 Categoria Administrativa
plt.figure(figsize=(12, 8))
sns.boxplot(x='Categoria Administrativa Descrição', y='Taxa de Desistência Acumulada - TDA', 
            data=df_clean)
plt.title('Taxa de Desistência Acumulada por Categoria Administrativa', fontsize=14)
plt.xlabel('Categoria Administrativa', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('figuras/desistencia_por_categoria.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.3.2 Modalidade de Ensino
plt.figure(figsize=(10, 6))
sns.boxplot(x='Modalidade de Ensino Descrição', y='Taxa de Desistência Acumulada - TDA', 
            data=df_clean)
plt.title('Taxa de Desistência Acumulada por Modalidade de Ensino', fontsize=14)
plt.xlabel('Modalidade de Ensino', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=12)
plt.tight_layout()
plt.savefig('figuras/desistencia_por_modalidade.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.3.3 Grau Acadêmico
plt.figure(figsize=(10, 6))
sns.boxplot(x='Grau Acadêmico Descrição', y='Taxa de Desistência Acumulada - TDA', 
            data=df_clean)
plt.title('Taxa de Desistência Acumulada por Grau Acadêmico', fontsize=14)
plt.xlabel('Grau Acadêmico', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=12)
plt.tight_layout()
plt.savefig('figuras/desistencia_por_grau.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.3.4 Região Geográfica
plt.figure(figsize=(12, 6))
sns.boxplot(x='Região Geográfica', y='Taxa de Desistência Acumulada - TDA', 
            data=df_clean)
plt.title('Taxa de Desistência Acumulada por Região', fontsize=14)
plt.xlabel('Região Geográfica', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=12)
plt.tight_layout()
plt.savefig('figuras/desistencia_por_regiao.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.4 Análise de desistência por coorte de ingresso
# Agrupando por ano de ingresso
desistencia_por_ingresso = df_clean.groupby('Ano de Ingresso')['Taxa de Desistência Acumulada - TDA'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='Ano de Ingresso', y='Taxa de Desistência Acumulada - TDA', 
            data=desistencia_por_ingresso)
plt.title('Taxa de Desistência Acumulada Média por Ano de Ingresso', fontsize=14)
plt.xlabel('Ano de Ingresso', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.savefig('figuras/desistencia_por_ano_ingresso.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.5 Análise de cursos com maiores taxas de desistência
# Considerando apenas cursos com pelo menos 100 ingressantes para maior confiabilidade
cursos_grandes = df_clean[df_clean['Quantidade de Ingressantes no Curso'] >= 100]
desistencia_por_curso = cursos_grandes.groupby('Nome do Curso de Graduação').agg({
    'Taxa de Desistência Acumulada - TDA': 'mean',
    'Quantidade de Ingressantes no Curso': 'sum'
}).reset_index()

# Top 10 cursos com maiores taxas
top_desistencia = desistencia_por_curso.nlargest(10, 'Taxa de Desistência Acumulada - TDA')

plt.figure(figsize=(12, 8))
sns.barplot(x='Taxa de Desistência Acumulada - TDA', y='Nome do Curso de Graduação', 
            data=top_desistencia, orient='h')
plt.title('Top 10 Cursos com Maiores Taxas de Desistência Acumulada', fontsize=14)
plt.xlabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.ylabel('Curso', fontsize=12)
plt.tight_layout()
plt.savefig('figuras/top10_cursos_desistencia.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.6 Análise de cursos com menores taxas de desistência
top_permanencia = desistencia_por_curso.nsmallest(10, 'Taxa de Desistência Acumulada - TDA')

plt.figure(figsize=(12, 8))
sns.barplot(x='Taxa de Desistência Acumulada - TDA', y='Nome do Curso de Graduação', 
            data=top_permanencia, orient='h')
plt.title('Top 10 Cursos com Menores Taxas de Desistência Acumulada', fontsize=14)
plt.xlabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.ylabel('Curso', fontsize=12)
plt.tight_layout()
plt.savefig('figuras/top10_cursos_menor_desistencia.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.7 Relação entre prazo de integralização e desistência
plt.figure(figsize=(10, 6))
sns.boxplot(x='Prazo de Integralização em Anos', y='Taxa de Desistência Acumulada - TDA', 
            data=df_clean)
plt.title('Relação entre Prazo de Integralização e Taxa de Desistência', fontsize=14)
plt.xlabel('Prazo de Integralização (Anos)', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('figuras/desistencia_vs_integralizacao.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2.8 Análise dos dados brutos (contagens absolutas)
# Criando proporções a partir das contagens originais para cada ano de referência
df_clean['Total de Alunos'] = df_clean['Quantidade de Permanência no Curso no ano de referência'] + \
                            df_clean['Quantidade de Concluintes no Curso no ano de referência'] + \
                            df_clean['Quantidade de Desistência no Curso no ano de referência']

df_clean['Proporção de Desistência'] = df_clean['Quantidade de Desistência no Curso no ano de referência'] / df_clean['Total de Alunos']
df_clean['Proporção de Permanência'] = df_clean['Quantidade de Permanência no Curso no ano de referência'] / df_clean['Total de Alunos']
df_clean['Proporção de Conclusão'] = df_clean['Quantidade de Concluintes no Curso no ano de referência'] / df_clean['Total de Alunos']

# Visualizando as contagens absolutas ao longo do tempo
contagens_por_tempo = df_clean.groupby('Tempo no Curso').agg({
    'Quantidade de Permanência no Curso no ano de referência': 'sum',
    'Quantidade de Concluintes no Curso no ano de referência': 'sum',
    'Quantidade de Desistência no Curso no ano de referência': 'sum'
}).reset_index()

# Transformando para formato "long" para visualização
contagens_long = pd.melt(contagens_por_tempo, 
                        id_vars=['Tempo no Curso'],
                        value_vars=['Quantidade de Permanência no Curso no ano de referência',
                                  'Quantidade de Concluintes no Curso no ano de referência',
                                  'Quantidade de Desistência no Curso no ano de referência'],
                        var_name='Status', value_name='Quantidade')

# Simplificando os nomes dos status
contagens_long['Status'] = contagens_long['Status'].map({
    'Quantidade de Permanência no Curso no ano de referência': 'Permanência',
    'Quantidade de Concluintes no Curso no ano de referência': 'Conclusão',
    'Quantidade de Desistência no Curso no ano de referência': 'Desistência'
})

plt.figure(figsize=(12, 7))
sns.lineplot(x='Tempo no Curso', y='Quantidade', hue='Status', 
            data=contagens_long, marker='o', linewidth=2)
plt.title('Evolução das Contagens Absolutas ao Longo do Tempo no Curso', fontsize=14)
plt.xlabel('Anos no Curso', fontsize=12)
plt.ylabel('Quantidade de Alunos', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(contagens_long['Tempo no Curso'].unique())
plt.legend(title='Status do Aluno')
plt.tight_layout()
plt.savefig('figuras/contagens_absolutas_tempo.png', bbox_inches='tight', dpi=600)
#plt.show()

In [ ]:
# Análise estatística para comparação entre grupos categóricos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

print("Análise estatística comparativa entre diferentes agrupamentos")

# 1. Análise por Categoria Administrativa
print("\n1. Comparação entre Categorias Administrativas")

# Verificando as categorias disponíveis
print("Categorias Administrativas disponíveis:")
print(df_clean['Categoria Administrativa Descrição'].value_counts())

# Criando grupos para teste
grupos_admin = {}
for categoria in df_clean['Categoria Administrativa Descrição'].unique():
    if pd.notna(categoria):  # Ignorando valores NaN
        valores = df_clean[df_clean['Categoria Administrativa Descrição'] == categoria]['Taxa de Desistência Acumulada - TDA'].dropna()
        if len(valores) > 0:
            grupos_admin[categoria] = valores.values

# Realizando o teste estatístico
if len(grupos_admin) >= 2:
    if len(grupos_admin) == 2:
        # Teste t para dois grupos
        chaves = list(grupos_admin.keys())
        t_stat, p_value = stats.ttest_ind(grupos_admin[chaves[0]], grupos_admin[chaves[1]], equal_var=False)
        print(f"\nResultado do Teste t (comparando {chaves[0]} e {chaves[1]}):")
        print(f"Estatística t: {t_stat:.4f}")
        print(f"Valor p: {p_value:.8f}")
        print(f"Diferença significativa a 5%: {'Sim' if p_value < 0.05 else 'Não'}")
        
        # Médias dos grupos
        for categoria, valores in grupos_admin.items():
            print(f"Média para {categoria}: {np.mean(valores):.2f}")
    else:
        # ANOVA para mais de dois grupos
        anova_groups = list(grupos_admin.values())
        f_stat, p_value = stats.f_oneway(*anova_groups)
        print(f"\nResultado da ANOVA:")
        print(f"Estatística F: {f_stat:.4f}")
        print(f"Valor p: {p_value:.8f}")
        print(f"Pelo menos uma média é significativamente diferente (5%): {'Sim' if p_value < 0.05 else 'Não'}")
        
        # Médias dos grupos
        for categoria, valores in grupos_admin.items():
            print(f"Média para {categoria}: {np.mean(valores):.2f}")
        
        # Se a ANOVA for significativa, realizar testes post-hoc
        if p_value < 0.05:
            print("\nResultados do teste post-hoc de Tukey HSD:")
            from statsmodels.stats.multicomp import pairwise_tukeyhsd
            
            # Preparando dados para Tukey
            values = np.concatenate([valores for valores in grupos_admin.values()])
            labels = np.concatenate([[categoria] * len(valores) 
                                     for categoria, valores in grupos_admin.items()])
            
            # Realizando teste de Tukey
            tukey_results = pairwise_tukeyhsd(values, labels, alpha=0.05)
            print(tukey_results)

# Visualização
plt.figure(figsize=(12, 8))
admin_stats = df_clean.groupby('Categoria Administrativa Descrição')['Taxa de Desistência Acumulada - TDA'].agg(
    ['mean', 'std', 'count']).reset_index()
admin_stats['error'] = admin_stats['std'] / np.sqrt(admin_stats['count'])

# Ordenando por média para melhor visualização
admin_stats = admin_stats.sort_values('mean', ascending=False)

plt.bar(admin_stats['Categoria Administrativa Descrição'], admin_stats['mean'], 
       yerr=1.96*admin_stats['error'], capsize=5, color='skyblue', width=0.6)
plt.title('Taxa de Desistência por Categoria Administrativa', fontsize=14)
plt.xlabel('Categoria Administrativa', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adicionando valores nas barras
for i, v in enumerate(admin_stats['mean']):
    plt.text(i, v + 2, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.savefig('figuras/tda_por_categoria_administrativa.png', bbox_inches='tight', dpi=600)
#plt.show()

# 2. Análise por Região Geográfica
print("\n2. Comparação entre Regiões Geográficas")

# Verificando as regiões disponíveis
print("Regiões Geográficas disponíveis:")
print(df_clean['Região Geográfica'].value_counts())

# Criando grupos para teste
grupos_regiao = {}
for regiao in df_clean['Região Geográfica'].unique():
    if pd.notna(regiao):  # Ignorando valores NaN
        valores = df_clean[df_clean['Região Geográfica'] == regiao]['Taxa de Desistência Acumulada - TDA'].dropna()
        if len(valores) > 0:
            grupos_regiao[regiao] = valores.values

# Realizando o teste estatístico (ANOVA)
if len(grupos_regiao) >= 2:
    anova_groups = list(grupos_regiao.values())
    f_stat, p_value = stats.f_oneway(*anova_groups)
    print(f"\nResultado da ANOVA:")
    print(f"Estatística F: {f_stat:.4f}")
    print(f"Valor p: {p_value:.8f}")
    print(f"Pelo menos uma média é significativamente diferente (5%): {'Sim' if p_value < 0.05 else 'Não'}")
    
    # Médias dos grupos
    for regiao, valores in grupos_regiao.items():
        print(f"Média para {regiao}: {np.mean(valores):.2f}")
    
    # Se a ANOVA for significativa, realizar testes post-hoc
    if p_value < 0.05:
        print("\nResultados do teste post-hoc de Tukey HSD:")
        
        # Preparando dados para Tukey
        values = np.concatenate([valores for valores in grupos_regiao.values()])
        labels = np.concatenate([[regiao] * len(valores) 
                                 for regiao, valores in grupos_regiao.items()])
        
        # Realizando teste de Tukey
        tukey_results = pairwise_tukeyhsd(values, labels, alpha=0.05)
        print(tukey_results)

# Visualização
plt.figure(figsize=(12, 6))
regiao_stats = df_clean.groupby('Região Geográfica')['Taxa de Desistência Acumulada - TDA'].agg(
    ['mean', 'std', 'count']).reset_index()
regiao_stats['error'] = regiao_stats['std'] / np.sqrt(regiao_stats['count'])

# Ordenando por média para melhor visualização
regiao_stats = regiao_stats.sort_values('mean', ascending=False)

plt.bar(regiao_stats['Região Geográfica'], regiao_stats['mean'], 
       yerr=1.96*regiao_stats['error'], capsize=5, color='lightgreen', width=0.6)
plt.title('Taxa de Desistência por Região Geográfica', fontsize=14)
plt.xlabel('Região Geográfica', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adicionando valores nas barras
for i, v in enumerate(regiao_stats['mean']):
    plt.text(i, v + 2, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.savefig('figuras/tda_por_regiao_geografica.png', bbox_inches='tight', dpi=600)
#plt.show()

# 3. Análise por Grau Acadêmico
print("\n3. Comparação entre Graus Acadêmicos")

# Verificando os graus disponíveis
print("Graus Acadêmicos disponíveis:")
print(df_clean['Grau Acadêmico Descrição'].value_counts())

# Criando grupos para teste
grupos_grau = {}
for grau in df_clean['Grau Acadêmico Descrição'].unique():
    if pd.notna(grau):  # Ignorando valores NaN
        valores = df_clean[df_clean['Grau Acadêmico Descrição'] == grau]['Taxa de Desistência Acumulada - TDA'].dropna()
        if len(valores) > 0:
            grupos_grau[grau] = valores.values

# Realizando o teste estatístico
if len(grupos_grau) >= 2:
    if len(grupos_grau) == 2:
        # Teste t para dois grupos
        chaves = list(grupos_grau.keys())
        t_stat, p_value = stats.ttest_ind(grupos_grau[chaves[0]], grupos_grau[chaves[1]], equal_var=False)
        print(f"\nResultado do Teste t (comparando {chaves[0]} e {chaves[1]}):")
        print(f"Estatística t: {t_stat:.4f}")
        print(f"Valor p: {p_value:.8f}")
        print(f"Diferença significativa a 5%: {'Sim' if p_value < 0.05 else 'Não'}")
        
        # Médias dos grupos
        for grau, valores in grupos_grau.items():
            print(f"Média para {grau}: {np.mean(valores):.2f}")
    else:
        # ANOVA para mais de dois grupos
        anova_groups = list(grupos_grau.values())
        f_stat, p_value = stats.f_oneway(*anova_groups)
        print(f"\nResultado da ANOVA:")
        print(f"Estatística F: {f_stat:.4f}")
        print(f"Valor p: {p_value:.8f}")
        print(f"Pelo menos uma média é significativamente diferente (5%): {'Sim' if p_value < 0.05 else 'Não'}")
        
        # Médias dos grupos
        for grau, valores in grupos_grau.items():
            print(f"Média para {grau}: {np.mean(valores):.2f}")
        
        # Se a ANOVA for significativa, realizar testes post-hoc
        if p_value < 0.05:
            print("\nResultados do teste post-hoc de Tukey HSD:")
            
            # Preparando dados para Tukey
            values = np.concatenate([valores for valores in grupos_grau.values()])
            labels = np.concatenate([[grau] * len(valores) 
                                     for grau, valores in grupos_grau.items()])
            
            # Realizando teste de Tukey
            tukey_results = pairwise_tukeyhsd(values, labels, alpha=0.05)
            print(tukey_results)

# Visualização
plt.figure(figsize=(10, 6))
grau_stats = df_clean.groupby('Grau Acadêmico Descrição')['Taxa de Desistência Acumulada - TDA'].agg(
    ['mean', 'std', 'count']).reset_index()
grau_stats['error'] = grau_stats['std'] / np.sqrt(grau_stats['count'])

# Ordenando por média para melhor visualização
grau_stats = grau_stats.sort_values('mean', ascending=False)

plt.bar(grau_stats['Grau Acadêmico Descrição'], grau_stats['mean'], 
       yerr=1.96*grau_stats['error'], capsize=5, color='salmon', width=0.6)
plt.title('Taxa de Desistência por Grau Acadêmico', fontsize=14)
plt.xlabel('Grau Acadêmico', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adicionando valores nas barras
for i, v in enumerate(grau_stats['mean']):
    plt.text(i, v + 2, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.savefig('figuras/tda_por_grau_academico.png', bbox_inches='tight', dpi=600)
#plt.show()

3. Análise Estatística das Taxas de Desistência


In [ ]:
# Parte 3 revisada: Análise Estatística Focada nas Dimensões Viáveis

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

# 3.1 Diagnóstico aprofundado das variáveis categóricas
print("Diagnóstico detalhado das variáveis categóricas:")
for coluna in df_clean.columns:
    if coluna in ['Categoria Administrativa Descrição', 'Modalidade de Ensino Descrição', 
                  'Grau Acadêmico Descrição', 'Região Geográfica']:
        n_unicos = df_clean[coluna].nunique()
        valores = df_clean[coluna].unique()
        print(f"\n{coluna}:")
        print(f"  Número de valores únicos: {n_unicos}")
        print(f"  Valores: {valores}")
        print(f"  Contagens:")
        print(df_clean[coluna].value_counts())

# 3.2 Análise Temporal das Taxas de Desistência (esta análise funcionou no código anterior)
print("\n3.2 Análise da Evolução da Taxa de Desistência ao Longo do Tempo no Curso")

# Certificando-se de que temos dados numéricos
df_clean['Taxa de Desistência Acumulada - TDA'] = pd.to_numeric(df_clean['Taxa de Desistência Acumulada - TDA'], errors='coerce')
df_clean['Tempo no Curso'] = pd.to_numeric(df_clean['Tempo no Curso'], errors='coerce')

# Removendo valores não numéricos
df_analysis = df_clean.dropna(subset=['Taxa de Desistência Acumulada - TDA', 'Tempo no Curso'])

# Agrupando por tempo no curso
evolucao_tempo = df_analysis.groupby('Tempo no Curso')['Taxa de Desistência Acumulada - TDA'].agg(
    ['mean', 'std', 'count']).reset_index()
evolucao_tempo['se'] = evolucao_tempo['std'] / np.sqrt(evolucao_tempo['count'])
evolucao_tempo['ci_lower'] = evolucao_tempo['mean'] - 1.96 * evolucao_tempo['se']
evolucao_tempo['ci_upper'] = evolucao_tempo['mean'] + 1.96 * evolucao_tempo['se']

plt.figure(figsize=(12, 6))
plt.errorbar(x=evolucao_tempo['Tempo no Curso'], y=evolucao_tempo['mean'], 
             yerr=1.96 * evolucao_tempo['se'], fmt='o-', capsize=5, 
             elinewidth=2, markeredgewidth=2)
plt.title('Evolução da Taxa de Desistência ao Longo do Tempo no Curso', fontsize=14)
plt.xlabel('Anos no Curso', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(evolucao_tempo['Tempo no Curso'].unique())
plt.tight_layout()
plt.savefig('figuras/evolucao_taxa_desistencia_tempo.png', bbox_inches='tight', dpi=600)
#plt.show()

# 3.3 Análise Longitudinal por Cohort (esta análise também funcionou)
print("\n3.3 Análise Longitudinal da Taxa de Desistência por Cohort de Ingresso")

# Verificando se temos a coluna Ano de Ingresso como numérica
df_analysis['Ano de Ingresso'] = pd.to_numeric(df_analysis['Ano de Ingresso'], errors='coerce')
df_analysis = df_analysis.dropna(subset=['Ano de Ingresso'])

# Agrupando por ano de ingresso e tempo no curso
cohort_data = df_analysis.groupby(['Ano de Ingresso', 'Tempo no Curso'])[
    'Taxa de Desistência Acumulada - TDA'].mean().reset_index()

# Criando heatmap para visualizar padrões
cohort_pivot = cohort_data.pivot(index='Ano de Ingresso', 
                                columns='Tempo no Curso', 
                                values='Taxa de Desistência Acumulada - TDA')

# Visualização do heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(cohort_pivot, annot=True, fmt=".1f", cmap="YlOrRd", linewidths=.5)
plt.title('Taxa de Desistência Acumulada por Cohort e Tempo no Curso', fontsize=14)
plt.xlabel('Anos no Curso', fontsize=12)
plt.ylabel('Ano de Ingresso', fontsize=12)
plt.tight_layout()
plt.savefig('figuras/heatmap_desistencia_cohort.png', bbox_inches='tight', dpi=600)
#plt.show()

# 3.4 Análise da Taxa de Desistência no Primeiro Ano por Cohort de Ingresso
print("\n3.4 Análise da Taxa de Desistência no Primeiro Ano")

# Filtrando dados do primeiro ano
primeiro_ano = df_analysis[df_analysis['Tempo no Curso'] == 1]
desistencia_primeiro_ano = primeiro_ano.groupby('Ano de Ingresso')[
    'Taxa de Desistência Acumulada - TDA'].agg(['mean', 'std', 'count']).reset_index()
desistencia_primeiro_ano['se'] = desistencia_primeiro_ano['std'] / np.sqrt(desistencia_primeiro_ano['count'])

plt.figure(figsize=(12, 6))
plt.errorbar(x=desistencia_primeiro_ano['Ano de Ingresso'], 
             y=desistencia_primeiro_ano['mean'], 
             yerr=1.96 * desistencia_primeiro_ano['se'], 
             fmt='o-', capsize=5, elinewidth=2, markeredgewidth=2)
plt.title('Taxa de Desistência no Primeiro Ano por Cohort de Ingresso', fontsize=14)
plt.xlabel('Ano de Ingresso', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('figuras/desistencia_primeiro_ano.png', bbox_inches='tight', dpi=600)
#plt.show()

# 3.5 Análise Avançada - Modelo de Regressão Simples para o Tempo no Curso
print("\n3.5 Modelo de Regressão Simples - Impacto do Tempo no Curso na Taxa de Desistência")

# Criando um modelo simples
X = sm.add_constant(df_analysis['Tempo no Curso'])
y = df_analysis['Taxa de Desistência Acumulada - TDA']

modelo = sm.OLS(y, X).fit()
print(modelo.summary())

# Visualizando a relação com linha de regressão
plt.figure(figsize=(10, 6))
sns.regplot(x='Tempo no Curso', y='Taxa de Desistência Acumulada - TDA', 
            data=df_analysis, scatter_kws={'alpha':0.3}, line_kws={'color':'red'})
plt.title('Relação entre Tempo no Curso e Taxa de Desistência', fontsize=14)
plt.xlabel('Anos no Curso', fontsize=12)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('figuras/regressao_tempo_desistencia.png', bbox_inches='tight', dpi=600)
#plt.show()

# 3.6 Comparação das Taxas de Desistência em Diferentes Anos do Curso (Abordagem Alternativa)
print("\n3.6 Comparação das Taxas de Desistência em Diferentes Anos do Curso")

# Usar pandas diretamente para análise estatística básica
tempo_stats = df_clean.groupby('Tempo no Curso')['Taxa de Desistência Acumulada - TDA'].agg([
    'mean', 'median', 'std', 'count'
]).reset_index()

# Calcular erro padrão e intervalos de confiança
tempo_stats['error'] = tempo_stats['std'] / np.sqrt(tempo_stats['count'])
tempo_stats['ci_lower'] = tempo_stats['mean'] - 1.96 * tempo_stats['error']
tempo_stats['ci_upper'] = tempo_stats['mean'] + 1.96 * tempo_stats['error']

print("Estatísticas da Taxa de Desistência por Ano no Curso:")
print(tempo_stats)

# Visualização da comparação entre anos
plt.figure(figsize=(14, 7))

# Gráfico de barras com barras de erro
plt.bar(tempo_stats['Tempo no Curso'], tempo_stats['mean'], 
        yerr=1.96*tempo_stats['error'], capsize=5, 
        color='skyblue', alpha=0.7, width=0.6)

# Melhorando o gráfico
plt.title('Evolução da Taxa de Desistência Acumulada por Ano no Curso', fontsize=16)
plt.xlabel('Anos no Curso', fontsize=14)
plt.ylabel('Taxa de Desistência Acumulada Média (%)', fontsize=14)
plt.xticks(tempo_stats['Tempo no Curso'], fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adicionando valores nas barras
for i, v in enumerate(tempo_stats['mean']):
    plt.text(i, v + 2, f'{v:.1f}%', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('figuras/comparacao_desistencia_anos_curso.png', bbox_inches='tight', dpi=600)
#plt.show()

# Teste ANOVA manual (sem usar a API de fórmula)
try:
    # Criando uma lista de grupos para análise
    grupos = []
    for ano in sorted(df_clean['Tempo no Curso'].unique()):
        # Obtendo os valores para cada ano do curso
        valores = df_clean[df_clean['Tempo no Curso'] == ano]['Taxa de Desistência Acumulada - TDA'].values
        if len(valores) > 0:  # Garantindo que temos dados
            grupos.append(valores)
    
    # Realizando o teste ANOVA manualmente se tivermos pelo menos 2 grupos
    if len(grupos) >= 2:
        from scipy import stats
        f_stat, p_value = stats.f_oneway(*grupos)
        print(f"\nResultados da ANOVA (manualmente calculada):")
        print(f"Estatística F: {f_stat:.4f}")
        print(f"Valor p: {p_value:.8f}")
        print(f"Significativo a 5%: {'Sim' if p_value < 0.05 else 'Não'}")
    else:
        print("Não há grupos suficientes para realizar o teste ANOVA.")
except Exception as e:
    print(f"Erro no cálculo manual da ANOVA: {e}")

# Teste de Kruskal-Wallis (alternativa não-paramétrica à ANOVA)
try:
    if len(grupos) >= 2:
        h_stat, p_value = stats.kruskal(*grupos)
        print(f"\nResultados do Teste Kruskal-Wallis (alternativa não-paramétrica):")
        print(f"Estatística H: {h_stat:.4f}")
        print(f"Valor p: {p_value:.8f}")
        print(f"Significativo a 5%: {'Sim' if p_value < 0.05 else 'Não'}")
except Exception as e:
    print(f"Erro no teste Kruskal-Wallis: {e}")

# Visualização mais detalhada: boxplot
plt.figure(figsize=(14, 8))
# Convertendo para categoria para garantir a ordem correta
df_clean['Ano no Curso'] = pd.Categorical(df_clean['Tempo no Curso'].astype(str), 
                                         categories=[str(x) for x in sorted(df_clean['Tempo no Curso'].unique())])

sns.boxplot(x='Ano no Curso', y='Taxa de Desistência Acumulada - TDA', data=df_clean)
plt.title('Distribuição da Taxa de Desistência por Ano no Curso', fontsize=16)
plt.xlabel('Ano no Curso', fontsize=14)
plt.ylabel('Taxa de Desistência Acumulada (%)', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.savefig('figuras/boxplot_desistencia_por_ano_curso.png', bbox_inches='tight', dpi=600)
#plt.show()